# Preprocessing

> Data preprocessing utilities.

In [ ]:
#| default_exp preprocessing

In [ ]:
#| export
import fastcore.all as fc
from pathlib import Path
from sklearn.base import BaseEstimator, TransformerMixin
from typing import List
import numpy as np
import pywt

In [ ]:
#|export
class MeanCenter(BaseEstimator, TransformerMixin):
    "Mean center spectra."
    def fit(self, X, y=None): return self
    def transform(self, X, y=None):     
        return X - X.mean(axis=1, keepdims=True) 

In [ ]:
#|export
class MeanReplicates(BaseEstimator, TransformerMixin):
    "Select replicated of specified spectra and average their spiked versions."
    def __init__(self, 
                 smp_name:str, # Sample of interest: 'LUI', 'SPA1' or 'TM4.1'.
                 names:List, # Names of scanned samples and replicates.
                ):        
        fc.store_attr()
        
    def _mean(self, X, substring):
        return X[[substring in name for name in self.names],:].mean(axis=0)
    
    def fit(self, X, y=None): return self
        
    def transform(self, X, y=None):     
        X =  np.array([self._mean(X, f'{self.smp_name}-{idx}') for idx in [0, 1, 2, 3]])
        return X - X.mean(axis=1, keepdims=True)

In [ ]:
t = list(range(10))

In [ ]:
t[0]

0

In [ ]:
filtered_levels = [0, -1, -2]

In [ ]:
#[0, 
#                      len(coeffs) - 1, 
#                      len(coeffs) - 2, 
#                      len(coeffs) - 3
#                     ]

In [ ]:
[level if level >=0 else (len(t) + level)  for level in filtered_levels]

[0, 9, 8]

In [ ]:
#|export
class DWTFiltering(BaseEstimator, TransformerMixin):
    def __init__(self, wavelet='db4', filtered_levels=[0, -1, -2]):
        fc.store_attr()

    def fit(self, X, y=None): return self
    def transform(self, X, y=None):     
        X_filtered = []
        for spectra in X:
            coeffs = pywt.wavedec(spectra, self.wavelet, level=None, mode='smooth')
    
            levels = [level if level >=0 else (len(coeffs) + level)  for level in self.filtered_levels]
            
            # Set the coefficients to filter to zero
            for level in levels:
                coeffs[level] = np.zeros_like(coeffs[level])
            
            X_filtered.append(pywt.waverec(coeffs, self.wavelet, mode='smooth'))
        return np.array(X_filtered)

In [ ]:
#|export
class DiffFromUnspiked(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X, y=None): return X[1:] - X[0]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()